In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
import cv2
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import torch.nn as nn
from torchvision.models import vit_b_16
from torch.optim import Adam
import matplotlib.pyplot as plt
import kagglehub
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import ssl
from timm import create_model
from collections import Counter


In [ ]:
# Download latest version
path = kagglehub.dataset_download("immada/casia-fasd")

print("Path to dataset files:", path)

100%|██████████| 2.05G/2.05G [00:14<00:00, 154MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/immada/casia-fasd/versions/1


In [ ]:
import os
import random
from PIL import Image
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Dataset personalizado para o CASIA-FASD com balanceamento opcional
class CASIA_FASD_Dataset(Dataset):
    def __init__(self, base_dir, dataset_type="train", transform=None):
        self.base_dir = base_dir
        self.dataset_type = dataset_type
        self.transform = transform

        self.live_paths = []
        self.spoof_paths = []

        for label_dir in ["live", "spoof"]:
            label = 1 if label_dir == "live" else 0
            label_path = os.path.join(base_dir, dataset_type, label_dir)
            print(f"Verificando o diretório: {label_path}")
            for root, _, files in os.walk(label_path):
                for file in files:
                    if file.endswith('.png'):
                        full_path = os.path.join(root, file)
                        if label == 1:
                            self.live_paths.append((full_path, label))
                        else:
                            self.spoof_paths.append((full_path, label))

        self.refresh()

    def refresh(self):
        # Balanceamento das classes por undersampling
        min_len = min(len(self.live_paths), len(self.spoof_paths))
        print(f"[REFRESH] Balanceando para {min_len} imagens por classe nesta época")

        sampled_live = random.sample(self.live_paths, min_len)
        sampled_spoof = random.sample(self.spoof_paths, min_len)

        balanced_data = sampled_live + sampled_spoof
        random.shuffle(balanced_data)

        self.image_paths, self.labels = zip(*balanced_data)
        self.image_paths = list(self.image_paths)
        self.labels = list(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def __repr__(self):
        return f"CASIA_FASD_Dataset(dataset_type='{self.dataset_type}', num_samples={len(self)})"

# Caminho base do dataset
base_dir = r"/root/.cache/kagglehub/datasets/immada/casia-fasd/versions/1/casia-fasd"

# Transformações dos frames
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Criar Datasets e DataLoaders
train_dataset = CASIA_FASD_Dataset(base_dir, dataset_type="train", transform=transform)
test_dataset = CASIA_FASD_Dataset(base_dir, dataset_type="test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, num_workers=20, pin_memory=True, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, num_workers=20, pin_memory=True, shuffle=False)

# Exibir os tamanhos dos datasets
print(f"Tamanho do Dataset de Treino: {len(train_dataset)}")
print(f"Tamanho do Dataset de Teste: {len(test_dataset)}")

train_labels = train_dataset.labels
test_labels = test_dataset.labels

print("Distribuição no Treino:", Counter(train_labels))
print("Distribuição no Teste:", Counter(test_labels))


Verificando o diretório: /root/.cache/kagglehub/datasets/immada/casia-fasd/versions/1/casia-fasd/train/live
Verificando o diretório: /root/.cache/kagglehub/datasets/immada/casia-fasd/versions/1/casia-fasd/train/spoof
[REFRESH] Balanceando para 19011 imagens por classe nesta época
Verificando o diretório: /root/.cache/kagglehub/datasets/immada/casia-fasd/versions/1/casia-fasd/test/live
Verificando o diretório: /root/.cache/kagglehub/datasets/immada/casia-fasd/versions/1/casia-fasd/test/spoof
[REFRESH] Balanceando para 10128 imagens por classe nesta época
Tamanho do Dataset de Treino: 38022
Tamanho do Dataset de Teste: 20256
Distribuição no Treino: Counter({0: 19011, 1: 19011})
Distribuição no Teste: Counter({0: 10128, 1: 10128})


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    matthews_corrcoef
)
import ssl
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms
from tqdm import tqdm
import numpy as np
from timm import create_model

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modelo Vision Transformer (ViT) para classificação binária
model = create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)
model = model.to(device)

# Função de custo com pesos
weights = torch.tensor([1.0, 1.0], dtype=torch.float).to(device)  # Ajuste os pesos conforme necessário.
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = Adam(model.parameters(), lr=1e-6)

# Avaliação do modelo
def evaluate_model(model, dataloader, device, full_eval=False):
    model.eval()
    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs[:, 1].cpu().numpy())

    y_true, y_pred, y_probs = np.array(y_true), np.array(y_pred), np.array(y_probs)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"Acurácia: {acc:.4f}\nPrecisão: {prec:.4f}\nRecall: {rec:.4f}\nF1-score: {f1:.4f}")

    if full_eval:
        auc = roc_auc_score(y_true, y_probs)
        mcc = matthews_corrcoef(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred)
        print(f"AUC-ROC: {auc:.4f}\nMCC: {mcc:.4f}\nMatriz de Confusão:\n{cm}")

        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp)
        print(f"Specificidade: {specificity:.4f}\n")
        print("Relatório de Classificação:")
        print(classification_report(y_true, y_pred, target_names=["spoof", "live"]))

    model.train()
    return acc

# Treinamento com salvamento do melhor modelo
best_acc = 0
best_model_path = "melhor_modelo_vit.pth"
num_epochs = 20

for epoch in range(num_epochs):
    train_dataset.refresh()
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 5 == 0:
        print(f"\nAvaliação após Epoch {epoch+1}:")
        train_acc = evaluate_model(model, train_loader, device, full_eval=False)

        val_acc = evaluate_model(model, test_loader, device, full_eval=False)
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"\nNovo melhor modelo salvo com acurácia: {best_acc:.4f}")

print("\nAvaliação Final no Conjunto de Teste:")
model.load_state_dict(torch.load(best_model_path))
evaluate_model(model, test_loader, device, full_eval=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

[REFRESH] Balanceando para 19011 imagens por classe nesta época


Epoch 1/20: 100%|██████████| 298/298 [04:04<00:00,  1.22it/s]


[REFRESH] Balanceando para 19011 imagens por classe nesta época


...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Acurácia: 0.9894
Precisão: 0.9976
Recall: 0.9811
F1-score: 0.9893

Novo melhor modelo salvo com acurácia: 0.9894

Avaliação Final no Conjunto de Teste:


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Acurácia: 0.9894
Precisão: 0.9976
Recall: 0.9811
F1-score: 0.9893
AUC-ROC: 0.9997
MCC: 0.9789
Matriz de Confusão:
[[10104    24]
 [  191  9937]]
Specificidade: 0.9976

Relatório de Classificação:
              precision    recall  f1-score   support

       spoof       0.98      1.00      0.99     10128
        live       1.00      0.98      0.99     10128

    accuracy                           0.99     20256
   macro avg       0.99      0.99      0.99     20256
weighted avg       0.99      0.99      0.99     20256



0.9893858609794629